In [1]:
using Convex
import PyPlot

INFO: Precompiling module Convex.

Use "abstract type Vexity end" instead.

Use "abstract type Monotonicity end" instead.

Use "abstract type Sign end" instead.

Use "abstract type AbstractExpr end" instead.

Use "abstract type Constraint end" instead.

Use "const Value = Union{Number,AbstractArray}" instead.

Use "const ValueOrNothing = Union{Value,Void}" instead.

Use "const AbstractExprOrValue = Union{AbstractExpr,Value}" instead.
ERROR: LoadError: LoadError: The default solver is set to `nothing`
         You must have at least one solver installed to use Convex.
         You can install a solver such as SCS by running:
         Pkg.add("SCS").
         You will have to restart Julia after that.
Stacktrace:
 [1] get_default_solver at /Users/kyle/.julia/v0.6/Convex/src/solver_info.jl:12 [inlined]
 [2] anonymous at ./<missing>:?
 [3] include_from_node1(::String) at ./loading.jl:569
 [4] include(::String) at ./sysimg.jl:14
 [5] include_from_node1(::String) at ./loading.jl:569
 [6] inc

LoadError: [91mFailed to precompile Convex to /Users/kyle/.julia/lib/v0.6/Convex.ji.[39m

In [2]:
Pkg.add("SCS")

INFO: Cloning cache of SCS from https://github.com/JuliaOpt/SCS.jl.git
INFO: Installing SCS v0.3.1
INFO: Building Homebrew
INFO: Building SCS
INFO: Package database updated


In [2]:
# spline control points
Pts = 50*[0 -1 -2 -3 -3.5 -3 -2 -1 -0.5 -1 -2 -3 -4 -5 -6 -6.5 -6 -5.5 -6 -6 -5 -4 -3 -2 -1.5 -1 0 1 1.5 2 3 4 5 6.25 7 7 7 7 7 7 7 6 5 4 4 4 3 2 1 0; 
       0 0 0 0 -1 -2 -2 -2 -3 -4 -4 -4 -4 -4 -4 -3 -2 -1 0 1 2 3 4 4 3 2 2 2 3 4 4 4 4 4 3 2 1 0 -1 -2 -3 -4 -4 -3 -2 -1 0 0 0 0]
degree = 3 # degree of spline
num_points = 10001
num_samples = 420

track = Raceway(Pts,degree,num_points,num_samples,lane_width)
track;

LoadError: UndefVarError: Raceway not defined

In [ ]:
# MPC
T # planning horizon
v₀ = 10.0 # fixed velocity for this particular problem
L₁ = 2.0
L₂ = 2.0

# control inputs
δ = Variable(T)
a = Variable(T)

# state variables
v = Variable(T+1)
ψ = Variable(T+1)
β = Variable(T+1)
ẋ = Variable(T+1)
ẏ = Variable(T+1)
ṡ = Variable(T+1)

s = Variable(T+1)
θ = Variable(T+1)
n = Variable(T+1)
C = Variable(T+1)
ϕ = Variable(T+1)

# state constraints
constraints = [
    # equality constraints (dynamics)
    v[1:end] == v₀
    ẋ == v[1:end-1] * cos(ψ[1:end-1] + β[1:end-1])
    ẏ == v[1:end-1] * sin(ψ[1:end-1] + β[1:end-1])
    Sf == ( 1 - n*C) / (ẋ*cos(ϕ) + ẏ*sin(ϕ))
    x[2:end] == ẋ[1:end-1] * Sf[1:end-1]
    y[2:end] == ẏ[1:end-1] * Sf[1:end-1]
    # s[2:end] == KDProject(x,y)
]

In [2]:
# Generate random problem data
m = 4;  n = 5
A = randn(m, n); b = randn(m, 1)

# Create a (column vector) variable of size n x 1.
x = Variable(n)

# The problem is to minimize ||Ax - b||^2 subject to x >= 0
# This can be done by: minimize(objective, constraints)
problem = minimize(sumsquares(A * x - b), [x >= 0])

# Solve the problem by calling solve!
solve!(problem)

# Check the status of the problem
problem.status # :Optimal, :Infeasible, :Unbounded etc.

# Get the optimal value
problem.optval

----------------------------------------------------------------------------
	SCS v1.2.6 - Splitting Conic Solver
	(c) Brendan O'Donoghue, Stanford University, 2012-2016
----------------------------------------------------------------------------
Lin-sys: sparse-direct, nnz in A = 31
eps = 1.00e-04, alpha = 1.80, max_iters = 20000, normalize = 1, scale = 5.00
Variables n = 8, constraints m = 15
Cones:	primal zero / dual free vars: 1
	linear vars: 6
	soc vars: 8, soc blks: 2
Setup time: 1.52e-03s
----------------------------------------------------------------------------
 Iter | pri res | dua res | rel gap | pri obj | dua obj | kap/tau | time (s)
----------------------------------------------------------------------------
     0|      inf       inf       nan      -inf       inf       inf  1.05e-03 
   100| 3.67e-06  3.53e-05  2.13e-05  1.32e+00  1.32e+00  3.19e-17  1.12e-03 
----------------------------------------------------------------------------
Status: Solved
Timing: Solve time: 

1.3238298064740333